In [4]:
## In this kernel I would like to show: 
## 1. FE creation approaches
## 2. Sequential fe validation
## 3. Dimension reduction
## 4. FE validation by Permutation importance
## 5. Mean encodings
## 6. Parallelization for FE

In [5]:
import numpy as np 
import pandas as pd 
import os, sys, gc, warnings, psutil, random

warnings.filterwarnings('ignore')

In [6]:
def timeseries_target_encoding(dt, year_cols, cols, func):
    year_target_dict = pd.DataFrame()
    for i, year in enumerate(year_cols):            
        if i  >= len(year_cols) - 1:
            break
        dt_group_tmp = dt[dt["year"] <= year].groupby(cols).agg(
            timeseries_target_encoding = ("sales", func)
        ).reset_index()
        dt_group_tmp = dt_group_tmp.rename(columns={"timeseries_target_encoding": f'enc_timeseries_{"_".join(cols)}_{func}'})
        year_target_dict = dt_group_tmp
        year_target_dict['year'] = year + 1
    dt = pd.merge(dt, year_target_dict, how="left", on=["year"] + cols)
    return dt
year_cols = [2011, 2012, 2013, 2014, 2015, 2016]

In [7]:
########################### Apply on grid_df
#################################################################################
# lets read grid from 
# https://www.kaggle.com/kyakovlev/m5-simple-fe
# to be sure that our grids are aligned by index

# データのパス変更の必要あり
# grid_df = pd.read_pickle('../data/preprocess/grid_part_1.pkl')
# calendar = pd.read_csv("../data/raw/calendar.csv")
TARGET = 'sales' 
grid_df = pd.read_pickle('../data/preprocess/grid_part_1_update.pkl')
calendar = pd.read_csv("../data/raw/calendar.csv")

calendar["d"] = calendar["d"].apply(lambda x : x.split("_")[1])
calendar["d"] = calendar["d"].astype("int16")
grid_df = pd.merge(grid_df, calendar[["d", "year"]], how="left", on="d")

grid_df[TARGET][grid_df['d']>(1913-28)] = np.nan
grid_df.loc[grid_df[TARGET]==0, TARGET] = np.nan
base_cols = list(grid_df)

icols =  [
            ['state_id'],
            ['store_id'],
            ['cat_id'],
            ['dept_id'],
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            ['item_id'],
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
            ]
"""
for col in icols:
    print('Encoding', col)
    col_name = '_'+'_'.join(col)+'_'
    grid_df['enc'+col_name+'mean'] = grid_df.groupby(col)[TARGET].transform('mean').astype(np.float16)
    grid_df['enc'+col_name+'std'] = grid_df.groupby(col)[TARGET].transform('std').astype(np.float16)
"""
for cols in icols:
    print('Encoding', cols)
    grid_df = timeseries_target_encoding(grid_df, year_cols, cols, "mean")
    grid_df = timeseries_target_encoding(grid_df, year_cols, cols, "std")



keep_cols = [col for col in list(grid_df) if col not in base_cols]
grid_df = grid_df[['id','d']+keep_cols]

Encoding ['state_id']
Encoding ['store_id']
Encoding ['cat_id']
Encoding ['dept_id']
Encoding ['state_id', 'cat_id']
Encoding ['state_id', 'dept_id']
Encoding ['store_id', 'cat_id']
Encoding ['store_id', 'dept_id']
Encoding ['item_id']
Encoding ['item_id', 'state_id']
Encoding ['item_id', 'store_id']


In [9]:
keep_cols

['enc_timeseries_state_id_mean',
 'enc_timeseries_state_id_std',
 'enc_timeseries_store_id_mean',
 'enc_timeseries_store_id_std',
 'enc_timeseries_cat_id_mean',
 'enc_timeseries_cat_id_std',
 'enc_timeseries_dept_id_mean',
 'enc_timeseries_dept_id_std',
 'enc_timeseries_state_id_cat_id_mean',
 'enc_timeseries_state_id_cat_id_std',
 'enc_timeseries_state_id_dept_id_mean',
 'enc_timeseries_state_id_dept_id_std',
 'enc_timeseries_store_id_cat_id_mean',
 'enc_timeseries_store_id_cat_id_std',
 'enc_timeseries_store_id_dept_id_mean',
 'enc_timeseries_store_id_dept_id_std',
 'enc_timeseries_item_id_mean',
 'enc_timeseries_item_id_std',
 'enc_timeseries_item_id_state_id_mean',
 'enc_timeseries_item_id_state_id_std',
 'enc_timeseries_item_id_store_id_mean',
 'enc_timeseries_item_id_store_id_std']

In [10]:
for col in keep_cols:
    grid_df[col] = grid_df[col].astype("float32")

In [11]:
grid_df

,id,d,enc_timeseries_state_id_mean,enc_timeseries_state_id_std,enc_timeseries_store_id_mean,enc_timeseries_store_id_std,enc_timeseries_cat_id_mean,enc_timeseries_cat_id_std,enc_timeseries_dept_id_mean,enc_timeseries_dept_id_std,...,enc_timeseries_store_id_cat_id_mean,enc_timeseries_store_id_cat_id_std,enc_timeseries_store_id_dept_id_mean,enc_timeseries_store_id_dept_id_std,enc_timeseries_item_id_mean,enc_timeseries_item_id_std,enc_timeseries_item_id_state_id_mean,enc_timeseries_item_id_state_id_std,enc_timeseries_item_id_store_id_mean,enc_timeseries_item_id_store_id_std
0,HOBBIES_1_008_CA_1_evaluation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_009_CA_1_evaluation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_010_CA_1_evaluation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_012_CA_1_evaluation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_015_CA_1_evaluation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47735392,FOODS_3_823_WI_3_evaluation,1969,3.457852,5.800511,3.730136,6.858336,4.365826,7.760155,5.04745,9.220328,...,4.675436,8.495997,5.595888,10.182152,2.609261,2.210701,1.863605,1.311845,1.967347,1.537186
47735393,FOODS_3_824_WI_3_evaluation,1969,3.457852,5.800511,3.730136,6.858336,4.365826,7.760155,5.04745,9.220328,...,4.675436,8.495997,5.595888,10.182152,1.636455,1.189343,1.512427,0.890406,1.506356,1.003160
47735394,FOODS_3_825_WI_3_evaluation,1969,3.457852,5.800511,3.730136,6.858336,4.365826,7.760155,5.04745,9.220328,...,4.675436,8.495997,5.595888,10.182152,1.845799,1.302737,1.806889,1.291860,1.961023,1.498272
47735395,FOODS_3_826_WI_3_evaluation,1969,3.457852,5.800511,3.730136,6.858336,4.365826,7.760155,5.04745,9.220328,...,4.675436,8.495997,5.595888,10.182152,2.124158,1.442854,2.379508,1.663015,1.982578,1.382853


In [12]:
#################################################################################
print('Save Mean/Std encoding')
grid_df.to_pickle('../data/preprocess/mean_encoding_df_update_nonleak.pkl')

Save Mean/Std encoding


In [18]:
########################### Final list of new features
#################################################################################
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46881677 entries, 0 to 46881676
Data columns (total 24 columns):
 #   Column                                Dtype   
---  ------                                -----   
 0   id                                    category
 1   d                                     int16   
 2   enc_timeseries_state_id_mean          float32 
 3   enc_timeseries_state_id_std           float32 
 4   enc_timeseries_store_id_mean          float32 
 5   enc_timeseries_store_id_std           float32 
 6   enc_timeseries_cat_id_mean            float32 
 7   enc_timeseries_cat_id_std             float32 
 8   enc_timeseries_dept_id_mean           float32 
 9   enc_timeseries_dept_id_std            float32 
 10  enc_timeseries_state_id_cat_id_mean   float32 
 11  enc_timeseries_state_id_cat_id_std    float32 
 12  enc_timeseries_state_id_dept_id_mean  float32 
 13  enc_timeseries_state_id_dept_id_std   float32 
 14  enc_timeseries_store_id_cat_id_mean   float32 
 